# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [27]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper1.py


In [28]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer1.py


In [29]:
# You can use this cell for other experiments: for example, for combiner.

## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [30]:
%%writefile mapper_sorter.py
# Your code for sort job here. Don't forget to use magic writefile

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    print "%d\t%s" % (count, word)

Overwriting mapper_sorter.py


In [31]:
%%writefile reducer_sorter.py

import sys

for line in sys.stdin:
    try:
        count, key = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    print "%s\t%d" % (key, count)

Overwriting reducer_sorter.py


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [60]:
%%bash

OUT_DIR="assignment1_"$(date +"%s%6N")
OUT_DIR2="assignment1_sorter_"$(date +"%s%6N")
#OUT_DIR="assignment1"
#OUT_DIR2="assignment1_next_out"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} ${OUT_DIR2} > /dev/null
#hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="With Sorter" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.map.output.key.field.separator=\t \
    -D mapreduce.partition.keycomparator.options=-k1,1nr \
    -files mapper_sorter.py,reducer_sorter.py \
    -mapper "python mapper_sorter.py" \
    -reducer "python reducer_sorter.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR}/part-00000 | sed -n '7p;8q'
hdfs dfs -cat ${OUT_DIR2}/part-00000 | sed -n '7p;8q'
#hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null

0,1,0	1
is	126420


rm: `assignment1_1550414126982088': No such file or directory
rm: `assignment1_sorter_1550414126983021': No such file or directory
19/02/17 14:35:29 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/17 14:35:29 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/17 14:35:30 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/17 14:35:30 INFO mapreduce.JobSubmitter: number of splits:2
19/02/17 14:35:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1550410596063_0015
19/02/17 14:35:30 INFO impl.YarnClientImpl: Submitted application application_1550410596063_0015
19/02/17 14:35:30 INFO mapreduce.Job: The url to track the job: http://08453e1f2c84:8088/proxy/application_1550410596063_0015/
19/02/17 14:35:30 INFO mapreduce.Job: Running job: job_1550410596063_0015
19/02/17 14:35:36 INFO mapreduce.Job: Job job_1550410596063_0015 running in uber mode : false
19/02/17 14:35:36 INFO mapreduce.Job:  map 0% reduce 0%
19/02/

A few outputs, just to check up on things.

In [57]:
%%bash

hdfs dfs -ls assignment1/
hdfs dfs -ls assignment1_next_out/

Found 9 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-17 14:28 assignment1/_SUCCESS
-rw-r--r--   1 jovyan supergroup     670229 2019-02-17 14:28 assignment1/part-00000
-rw-r--r--   1 jovyan supergroup     675659 2019-02-17 14:28 assignment1/part-00001
-rw-r--r--   1 jovyan supergroup     664543 2019-02-17 14:28 assignment1/part-00002
-rw-r--r--   1 jovyan supergroup     673944 2019-02-17 14:28 assignment1/part-00003
-rw-r--r--   1 jovyan supergroup     665191 2019-02-17 14:28 assignment1/part-00004
-rw-r--r--   1 jovyan supergroup     679162 2019-02-17 14:28 assignment1/part-00005
-rw-r--r--   1 jovyan supergroup     668021 2019-02-17 14:28 assignment1/part-00006
-rw-r--r--   1 jovyan supergroup     673764 2019-02-17 14:28 assignment1/part-00007
Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-17 14:29 assignment1_next_out/_SUCCESS
-rw-r--r--   1 jovyan supergroup    5370513 2019-02-17 14:29 assignment1_next_out/part-00000


In [58]:
%%bash

hdfs dfs -cat assignment1_next_out/part-00000 | head -10

the	822164
of	447464
and	342715
in	292354
to	241467
a	236225
is	126420
as	103301
for	91245
was	90336


cat: Unable to write to output stream.


In [46]:
%%bash

hdfs dfs -cat assignment1/part-00003 | head -10

...baby	1
0&\mbox	1
0,0	16
0,1,3	1
0,17	1
0,1\}^{m	2
0,1\}^{n}\times	1
0,20	1
0,8	2
0,\tfrac{1}{2}\pi	1


cat: Unable to write to output stream.
